Here is JINT code + adaptation to the real world.
# Contents


[**Racionale**](#proveThePoint)

**Hardware**
- [Decide flight platform and ros version](#flightPlatform)
- [Hardware Architecture](#hardwareArchitecture)

**Development Environment**
- [Method](#method)
- [Nvidia](#Nvidia)
- [Docker](#dockerFile)
- [Gazebo simulation](#gazebo)
    - [Try 4 -  octomap camera](#try4)
    - [Try 5 -  velodyne adaptation](#try5)
    - [Try 6 -  older px4 sitl from 2015](#try6)
    - [Try 7 -  hector quadrotor](#try7)
    - [Try 8 -  grvc + px4](#try8)
    - [Try 9 -  try5 ETHz onwards](#try9)
    
**Software**
- [Architecture](#architecture)
- [Ros Mysteries](#rosMysteries)
- [MavRos](#mavros)
- Implementation challenges

[**Experiment**](#experiment)
- [Simulation environment](#Simulation_environment)
- [Steps](#Steps)
- [Dataset](#dataset)
- [What to measure](#What_to_measure)
- [Thinkpad](#thinkpad)
- [Launching instructions](#LaunchingInstructions)

[**Implementation details**](#Implementation)

**Result analysis - Were there any improvements?**

The data to be used will be the flights in the testbed.

- RAM memory implications
- Computation time of paths

**Bibliography**(#References)



[test link](Paper A.ipynb)

<a id='proveThePoint'></a>
# Racionale

This approach adds the abitility to inspect any building, without any prior knowledge (not even on google maps), in all space of the 3 dimensions (not just ground or pillars or wall).

The final tests must prove it works on the target scale (oilrigs). This will be done through scaling the bootleneck points (distances, memory, sensor resolution). [study here](Paper C - Large structures test.ipynb#scaling) and  [calculation sheet](https://docs.google.com/spreadsheets/d/1UFneS__9-L0viD8dhHrdOTUGh9j_4SgYT0UNm2cHwcY/edit?usp=sharing)

Obstacle avoidance online.



<a id='flightPlatform'></a>
# Flight platform & software requirements

- **Payload** Point cloud sensor + camera for later mapping
- **Autopilot** Pixhawk 1 (for vicon compatibility)
- **Onboard CPU/GPU** UPboard
- **Vicon interface** is ROS_VRPN_CLIENT, working with Pixhawk 1 (Fran has experience)
- [**Gazebo out of the box**](http://gazebosim.org/tutorials?tut=ros_wrapper_versions#Gazebo7.xseries) 
    - Kinetic works with Gazebo 8 and Xenial [here](https://github.com/CentroEPiaggio/iliad_wiki/wiki/1.-ROS-Kinetic-and-Gazebo8-setup) is an installation guide
    - [Docker with PxHawk + Gazebo + mavros](https://dev.px4.io/en/test_and_ci/docker.html)
- **Ros version** (see above)
- **Docker** Likely yes 

[Excel for calculations](http://localhost:8888/?token=87db5327c113e4e5779dc262cbc3c08597b230f34cb1927c
)

<a id='hardwareArchitecture'></a>
# Hardware Architecture

![component_connection_diagram](attachment:component_connection_diagram)

<a id='method'></a>
### Method

1. Docker - Evironment encapsulation. Summarize initial setup needed to reproduce experiment. Enable generate easily a new machine. Open the possiblity of quick fresh install, having different versions without the added complexity of making sure the right version of each tools is being used while alternating among investigation lines.
1. Git - keep track of development
1. Jupyter notebook - keep track of investigation progress recording abandoned paths, dead ends and mistakes. 
1. Synthesis matrix - comparing each analyzed aspect of each possibility. Forgotten aspects show up and also with certain problems didn't come up at each point.
1. Record aditional setup needed to start experiment. (also jupyter)
1. Unit testing. Assert code stability. Record api usage. Record development focus.
1. Gazebo. More plugins for sensors.
1. Match the simulated architecture with the UAV architecture as closely as possible (example Real-time visual-inertial mapping, re-localization and planning onboard MAVs in unknown environments p4)


<a id='Nvidia'></a>
### Nvidia 

The nvidia GPU drivers make a mess with docker. The best option is to just burn the whole thing with fire.

To do so follow [this guy](https://askubuntu.com/questions/206283/how-can-i-uninstall-a-nvidia-driver-completely)

Blacklisting does not work (or boot)

To check what is being used when nvidia is installed between the intel and nvidia: `prime-select query`

When the installation is correct the command is no longer recognized.

But if you by some reason need to alternate among the two it's `prime-select nvidia|intel`
    
The open source driver made the omen laptop stop working at all for two times. Seemingly at random.

Without GPU there are no additional monitors.

<a id='dockerFile'></a>
### Docker file


Problems with the networking part of things. Especially the GUI interface. Gui will be used for development for visual check of path generated (Lazy Theta*) and understanding the patterns generated by automated exploration. 

On the other hand, the big contribution of docker would be to run simultaneously experiments with different parameters, if these need visual confirmation it's pointless. 

BIG QUESTIONS: 
- How do I automate the assertion of the validity of a run? 
- How do I make videos for dissemination? (this is actually easy, just run locally one that you want to tape)

###### Following [Network Setup tutorial](http://wiki.ros.org/ROS/NetworkSetup) in same container
This turn out to be harder than anticipated. Made a question in [ROS Answers](https://answers.ros.org/question/273281/unable-to-setup-docker-on-kinetic/). Progress:
- If I start roscore on one bash of the container, it is not found on another bash of the same container. Looking into environment variables --> it seems I need to do `export ROS_MASTER_URI=http://172.17.0.2:11311/'

Instructions so far, using garuda as the name of the container: 

1. Start the container 
```
sudo docker run --name garuda -it osrf/ros:kinetic-desktop-xenial
```
    
2. In the local machine find container's IP
```
docker inspect garuda | grep IP
```
    
3. Install minimum stuff and start roscore
```
apt-get update
apt-get install wget
wget http://packages.ros.org/ros.key -O - |  apt-key add -
apt-get update
apt-get install python-catkin-tools
export ROS_MASTER_URI=http://<ip>:11311/
roscore
```    
4. Go to a different terminal and start the listener
```
docker exec -it garuda bash
export ROS_MASTER_URI=http://<ip>:11311/
source /opt/ros/ROS_DISTRO/setup.bash
rosrun rospy_tutorials listener.py
```
5. Go to yet another different terminal and start the talker 
```
docker exec -it garuda bash
export ROS_MASTER_URI=http://<ip>:11311/
source /opt/ros/$ROS_DISTRO/setup.bash
rosrun rospy_tutorials talker.py
```

###### Setup communication as two networked ROS machines, one docker and the other normal desktop. Following [Network Setup tutorial](http://wiki.ros.org/ROS/NetworkSetup)
1. Start the container with [-P flag was used to automatically map any network port inside it to a random high port within an ephemeral port range on your Docker host](https://docs.docker.com/engine/userguide/networking/default_network/dockerlinks/)
```
sudo docker run --name garuda -P -it osrf/ros:kinetic-desktop-xenial
```

2. Same but after `export ROS_MASTER_URI` run `export ROS_IP=<container ip>`

3. Same but after `export ROS_MASTER_URI` run `export ROS_IP=<container ip>`

4. Same but in local machine. 
    - Roscore hears the message but issues the following warning `Couldn't find an AF_INET address for [mfaria-Workstation]`.
    - If additionally the ROS_IP is set to the local machine IP it does not work and no message appears on roscore. 

###### Docker-compose
From the PhD student that setup the farms at OSRF an [example to use docker with docker-compose](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64)

1. [Install docker compose](https://docs.docker.com/compose/install/#install-compose) 
```sudo curl -L https://github.com/docker/compose/releases/download/3.2.0/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose```
2. Create a file with the files on the [gist](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64) and go there
3. `docker-compose up`
The original example is in indigo. If this works will need to upgrade TODO
4. The containers can talk with each other perfectly and the host can ping all of them. Meaning the following instructions work:

```
ping master
ping talker
ping listener```

However listing topics does not:

```ROS_MASTER_URI=http://master:11311
rostopic list
ERROR: Unable to communicate with master!
```

This was done with composer version 1.16.0 instead of the intended 1.6.2. Will revert to correcto composer version and try again.

###### Possible options for setting up the network
- I did not understood why the previous fails, as the inner workings of AF_INET and hosts are still unclear. However the developer involved in creating Open Source Robotics Foundation's farms contibuted a way of [launching several containers with docker-composer](https://gist.github.com/ruffsl/4a24c26a1aa2cc733c64).
- [Q Change hosts file](https://answers.ros.org/question/254269/docker-ros-unable-to-communicate-with-master/)
- [Q Remove network isolation](https://answers.ros.org/question/250054/how-to-use-docker-on-a-host-together-with-non-docker-machines-ros/)
- [Q Use vm network ip in ROS_MASTER_URI instead of domain name, for VM](https://answers.ros.org/question/227934/roscore-on-virtual-machine/)
- Checkout the docs for [intel aero platform with docker](https://github.com/intel-aero/meta-intel-aero/wiki/08-Aero-Network-and-System-Administration#sysadmin-docker-ros)

###### Docker for px4 + gazebo

[Installation guide](https://dev.px4.io/en/test_and_ci/docker.html)
It is not a very good one.
So far I additionally had to do:
1. create a dedicated folder 
1. cd into it
1. `git clone https://github.com/PX4/Firmware.git`
1. only then start on the installation guide. The provided command was adapted into <pre><code>docker run -it --privileged -v \`pwd\`:\`pwd\`:rw -v /tmp/.X11-unix:/tmp/.X11-unix:ro -e DISPLAY=:0 -p 14556:14556/udp --name=px4_gazebo px4io/px4-dev-ros:2017-05-31 bash</code></pre>
1. Then leave to enter as root
<pre><code>exit
xhost +
docker exec -u 0 -it px4_gazebo bash</pre></code>
1. Go into the Firmware folder and continue the installation
<pre><code>make posix_sitl_default gazebo</pre></code>

Issue submitted to px4 [here](https://github.com/PX4/containers/issues/81)

Need to do `xhost +` every time, but after that gazebo works. As does everything else.

**Seemed final but had the mount permission issue for workspace**

Created Dockerfile with px4-dev-ros:2017-10-23 as base and px4 firmware tag v1.6.5 (on git)

Built with  `sudo docker build -t garuda_sitl .` while at _docker folder

To launch:
1. `xhost +`
2. `docker run -it --privileged -v /home/mfaria/ws_mavlink_grvcHal/:/home/user/ros_workspace:rw -v /tmp/.X11-unix:/tmp/.X11-unix:ro -e DISPLAY=:0 -p 14556:14556/udp --name=px4_gazebo garuda_sitl bash`
3. `exit`
4. `docker start px4_gazebo`
4. `docker exec -u 0 -it px4_gazebo bash`
5. Go into the Firmware folder and continue the installation `make posix_sitl_default gazebo`


###### Firmware by volume mount

- Removed clone from dockerfile
- Launching as `docker run -it --privileged \
    -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
    -e DISPLAY=:0 \
    --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
- Launching as `sudo docker run -it --privileged     -v /home/mfaria/Flying_Octomap:/ros_ws     -v /tmp/.X11-unix:/tmp/.X11-unix:ro     -e DISPLAY=:0     --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
- From docker px4io/px4-dev-ros tag 2017-12-08
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. `cd /home/mfaria/Flying_Octomap/src/_docker`
        1. Built with  `docker build -t garuda_sitl .`
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl px4io/px4-dev-ros:2017-12-08 /bin/bash`
        1. exit
        1. `docker start garuda_sitl`
        1. `docker exec -u 0 -it garuda_sitl bash`
    1. Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        2. `catkin_make`
    1. To launch gazebo: 
        1. `cd /Firmware`
        1. `make posix_sitl_default gazebo`
- From docker custom image garuda_sitl_v2 based on px4io/px4-dev-ros tag 2017-12-08
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. Build project:
        1. `catkin build` (should take around 3 min)
    1. To launch gazebo: 
        1. `cd /Firmware`
        1. `make posix_sitl_default gazebo`

<a id='gazebo'></a>
# Gazebo and px4 SITL and MavRos

[PixHawk Offboard mode](https://dev.px4.io/en/ros/offboard_control.html)

##### Overlaying for launch
[Overlaying docs](http://wiki.ros.org/catkin/Tutorials/workspace_overlaying)
[Launching gazebo and pc4 separatly docs](https://dev.px4.io/en/simulation/ros_interface.html)

** Try 1 **
1. Docker
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. Lauch docker `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/ws_mavlink_grvcHal/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build with overlay
    1. Build base = px4. Keep terminal
        1. `cd /Firmware`
        1. `no_sim=1 make posix_sitl_default gazebo`
    1. Build overlay
        1. New terminal (check launch docker)
        1. (First time) Make ros workspace:
            1. `source /opt/ros/kinetic/setup.bash`
            1. `cd /ros_ws`
            2. `catkin init`
        1. `catkin build` (should take around 3 min)
        1. Launch gazebo w/ iris
            1. `roslaunch gazebo_ros empty_world.launch world_name:=/Firmware/Tools/sitl_gazebo/worlds/iris.world`
        
** Try 2 **
1. Docker
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap_code/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace
    1. (First time) Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. `catkin build` (should take around 3 min)
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    1. `cd /Firmware`
    1. `make posix_sitl_default gazebo`
    1. Kill process (this was just for the make part, they apperantly complicated the toolchain at px4)
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
    1. `roslaunch px4 posix_sitl.launch`
1. New terminal
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. `cd /Firmware`
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`


** Try 3 - px4 lidar with lidar **

No good because is a point cloud in LaserScan format. This is kind of merged with try 5.

[Thread with lidar code from px4](https://github.com/PX4/Firmware/pull/6558)
`roslaunch px4 posix_sitl.launch vehicle:=iris_rplidar est:=lpe`


<a id='try4'></a>
** Try 4 - px4 support for octomap ** 

No good because is Point Cloud from camera & for fixed wing.

[Rotors Simulator setup](https://github.com/ethz-asl/rotors_simulator/wiki/Setting-up-the-RotorS-Simulator)
the latest launch seems to be at "Setting up RotorS as Fixed Wing HiL Simulation (Pixhawk, Mavros, Mavlink, QGC)" but it is fixed wing and camera

[pX4 OCTOMAP in simulation](https://dev.px4.io/en/simulation/gazebo_octomap.html) `roslaunch rotors_gazebo mav_hovering_example_with_vi_sensor.launch  mav_name:=firefly`

1. Docker
    2. Build docker image from _docker_vision
        1. In Dockerfile add dependencies 
        ```ros-kinetic-rqt-gui 
        ros-kinetic-rqt-gui-py 
        ros-kinetic-xacro 
        ros-kinetic-robot-state-publisher 
        ros-kinetic-joint-state-publisher```
        1. Also add commands:
        ```
        && mkdir /eth_ws \
        && mkdir /eth_ws/src \
        && cd /eth_ws/src \
        && git clone https://github.com/ethz-asl/rotors_simulator.git \
        && git clone https://github.com/ethz-asl/mav_comm.git \
        && git clone https://github.com/ethz-asl/glog_catkin.git \
        && git clone https://github.com/catkin/catkin_simple.git \
        ```
        1. Built with  `docker build -t garuda_sitl_vision .` while at _docker_vision folder
    3. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap_code/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_vision garuda_sitl_vision /bin/bash`
        1. exit
        1. `docker start garuda_sitl_vision`
        1. `docker exec -u 0 -it garuda_sitl_vision bash`
1. Build ETH workspace
    1. (First time) Make ETH workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /eth_ws`
        2. `catkin init`
    1. `catkin build` (should take around  min)
1. New terminal
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_vision bash`
    1. `cd /Firmware`
    1. `source /ros_ws/devel/setup.bash`
    1. `source /eth_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`



<a id='try5'></a>
** Try 5 - add velodyne to iris **


- Everything seems well (topic has messages, tf frame of messages is connected with the fized frame (base_link)) but no points are shown. [Gazebo question](https://community.gazebosim.org/t/rrbot-tutorial-gazebo-and-rviz/73) [Ros question](https://answers.ros.org/question/279283/pointcloud-from-velodyne-in-rviz/) [Px4 issue](https://github.com/PX4/sitl_gazebo/issues/159)
- Problem with tf tree [issue](https://github.com/PX4/sitl_gazebo/issues/159). Possible solution [converting sdf/xacro to urdf](http://answers.gazebosim.org/question/4192/problem-with-sdf-files-or-robot_description-parameter/) Needed to convert to urdf + robot_state_publisher + joint_state_publisher.
- Pay attention to the paths with not finding the plugin / model. [Environment Variables](http://gazebosim.org/tutorials?tut=components)

- Use the Velodyne sim. Output PointCloud2.

-  pandora / grvcQuadrotor / grvc_quadrotor_gazebo / urdf / grvc_quadrotor_base_velodyne.urdf.xacro 

```<VLP-16 parent="${base_frame}" name="velodyne_down" topic="/${model_name}/velodyne_points/down">
  <origin xyz="-0.1 -0.1 -0.2" rpy="0 0 0" />
</VLP-16>```

-  pandora / dependencies / velodyne_simulator / velodyne_description / urdf / VLP-16.urdf.xacro 
 
```<scan>
    <horizontal>
      <samples>500</samples>
      <resolution>1</resolution>
      <min_angle>-${M_PI}</min_angle>
      <max_angle> ${M_PI}</max_angle>
    </horizontal>
    <vertical>
      <samples>16</samples>
      <resolution>1</resolution>
      <min_angle>-${15.0*M_PI/180.0}</min_angle>
      <max_angle> ${15.0*M_PI/180.0}</max_angle>
    </vertical>
</scan>```

1. Docker
    1. Clone Firmware folder
        1. Download git `git clone https://github.com/PX4/Firmware.git`
        1. Change to tag `git checkout v1.7.0-rc4`
        1. [Deprecated] In file Firmaware /Tools/sitl_gazebo/models/rotors_description/urdf/iris.xacro Add at line 149 
        ```
        <xacro:include filename="/opt/ros/kinetic/share/velodyne_description/urdf/VLP-16.urdf.xacro"/> 
  <VLP-16 parent="base_link" name="velodyne" topic="/velodyne_points" hz="10" samples="440"> 
    <origin xyz="0 0 0.4" rpy="0 0 0" /> 
  </VLP-16>
  ```
    2. Build docker image
        1. Built with  `docker build -t garuda_sitl_v2 .` while at _docker folder
    3. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap_code:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace
    1. (First time) Make ros workspace:
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. `catkin build` (should take around 3 min)
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    1. `cd /Firmware`
    1. `make posix_sitl_default gazebo`
    1. Kill process (this was just for the make part, they apperantly complicated the toolchain at px4)
    1. Deprecated Change /Firmware/Tools/sitl_gazebo/models/iris/irirs.sdf At frameName change to local_origin
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
    1. `cp /ros_ws/src/architecture/Tools/iris.xacro /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp /ros_ws/src/architecture/Tools/iris_base.xacro /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp /ros_ws/src/architecture/Tools/posix_sitl_urdf.launch /Firmware/launch/`
    1. `python /Firmware/Tools/sitl_gazebo/scripts/xacro.py -o /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/Firmware/Tools/sitl_gazebo/models/rotors_description`
    1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. Change the vertuical angle of the rays in urdf `sed -i 's/_/_/g' /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. `roslaunch px4 posix_sitl_urdf.launch`
1. New terminal with ros_ws
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. `cd /Firmware`
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. New terminal just Firmware
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. `cd /Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
    1. `export PYTHONPATH=$PYTHONPATH:/usr/lib/python2.7/dist-packages` (rqt tools)
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`
    
```sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list' 
sudo apt-key adv --keyserver hkp://ha.pool.sks-keyservers.net:80 --recv-key 421C365BD9FF1F717815A3895523BAEEB01FA116 
wget http://packages.ros.org/ros.key -O - | sudo apt-key add - 
apt-get update ```

The rqt error is harmless so far. But in case it gets relevent [here is how to fix it](
https://github.com/activatedgeek/docker-videostack/issues/1)


<a id='try8'></a>
** Try 8 - grvc for px4 **

[grvc-ual@github](https://github.com/grvcTeam/grvc-ual)

[ros_control](http://gazebosim.org/tutorials/?tut=ros_control) is only for sending commands i.e. for the arms.

1. Get px4 code - Clone Firmware folder
    1. `mkdir ~/px4_grvc`
    1. `cd ~/px4_grvc`
    1. `git clone https://github.com/PX4/Firmware.git`
    1. `cd Firmware/`
    1. `git checkout -f v1.6.3`
    1. `git submodule update --recursive`
1. Docker
    1. At src/_docker_grvc/ folder, build docker image: `docker build -t garuda_sitl_grvc .` 
    1. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/px4_grvc/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap_code/src:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=$DISPLAY \
        --name=garuda_sitl_grvc garuda_sitl_grvc /bin/bash`
        1. exit
        1. `docker start garuda_sitl_grvc`
        1. `docker exec -u 0 -it garuda_sitl_grvc bash`
1. Build code inside container. Pay attention to the order of ROS_PACKAGE_PATH. Every time the setup.bash is sourced, the Firmware references are wipped out.
    1. `source /opt/ros/kinetic/setup.bash`
    1. `cd /Firmware`
    1. `make posix_sitl_default gazebo`
    1. kill
    1. `source /Firmware/Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `cd /ros_ws`
    1. `build init`
    1. `build catkin`
    1. `source devel/setup.bash`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. Launch simulation with grvc launch:  `roslaunch px4_bringup test_simulate.launch`
1. Launch rviz vizualization with grvc launch:  `roslaunch robots_description visualize.launch`
1. For new terminals
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_grvc bash`
    1. `source /Firmware/Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `cd /ros_ws`
    1. `source devel/setup.bash`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. In new terminal, the commands of architecture px4_mavros_offboard_node
    1. `rostopic pub /uav_1/mavros/setpoint_position/local geometry_msgs/PoseS`
    1. `rosservice call /uav_1/mavros/cmd/takeoff "{min_pitch: 0.0, yaw: 0.0, latitude: 0.0, longitude: 0.0, altitude: 5.0}" `
    1. `rosservice call /uav_1/mavros/cmd/arming "value: true"`
    1. `rosservice call /uav_1/mavros/set_mode "base_mode: 0 custom_mode: 'OFFBOARD'"`

```
std::string ual_ns =  this->ns_prefix_ + std::to_string(this->robot_id_) + "/ual";     
std::string take_off_srv = ual_ns + "/take_off";       
std::string land_srv = ual_ns + "/land";
std::string go_to_waypoint_srv = ual_ns + "/go_to_waypoint";
std::string go_to_waypoint_geo_srv = ual_ns + "/go_to_waypoint_geo";
std::string set_velocity_srv = ual_ns + "/set_velocity";
std::string set_position_error_srv = ual_ns + "/set_position_error";
std::string recover_from_manual_srv = ual_ns + "/recover_from_manual";
std::string pose_topic = ual_ns + "/pose";
std::string velocity_topic = ual_ns + "/velocity";

rosservice call /uav_1/ual/take_off "height: 2.0 blocking: false" 
rosservice call /uav_1/ual/go_to_waypoint_srv 
rosservice call /uav_1/ual/velocity_topic
```

<a id='try9'></a>
##### Try 9 - ETHz

This is after the winter school. Modifications found at ETHz.

Use the Velodyne sim. Output PointCloud2. (? not sure, Riccardo did this)
1. Clone repos
    1. Clone FlyingOctomap folder
        1. `git clone https://github.com/margaridaCF/FlyingOctomap_code.git`
        1. `git checkout ____`
        1. `mv FlyingOctomap_code Flying_Octomap_code`
    1. Clone Firmware folder
        1. `git clone https://github.com/PX4/Firmware.git`
        1. `cd Firmware/`
        1. `git checkout v1.7.0-rc4`
1. Docker
    2. Build docker image
        1. `cd /home/mfaria/Flying_Octomap_code/_docker`
        1. `docker build -t garuda_sitl_v2 .` 
    3. Launch docker
        1. `xhost +`
        1. `docker run -it --privileged \
        -v /home/mfaria/Firmware:/Firmware:rw \
        -v /home/mfaria/Flying_Octomap_code:/ros_ws/src:rw \
        -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
        -e DISPLAY=:0 \
        --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
        1. exit
        1. `docker start garuda_sitl_v2`
        1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace
    1. (First time) Make ros workspace:
        1. `cd /Firmware`
        1. `source /opt/ros/kinetic/setup.bash`
        1. `cd /ros_ws`
        2. `catkin init`
    1. `catkin build` 
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    1. `cd /Firmware`
    1. `wget https://raw.githubusercontent.com/mavlink/mavros/master/mavros/scripts/install_geographiclib_datasets.sh`
    1. `chmod +x install_geographiclib_datasets.sh`
    1. `./install_geographiclib_datasets.sh `
    1. `make posix_sitl_default gazebo`
    1. Kill process (this was just for the make part, they apperantly complicated the toolchain at px4)
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
    1. `cp /ros_ws/src/architecture/Tools/iris.xacro /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp /ros_ws/src/architecture/Tools/iris_base.xacro /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp /ros_ws/src/architecture/Tools/posix_sitl_urdf.launch /Firmware/launch/`
    1. `python /Firmware/Tools/sitl_gazebo/scripts/xacro.py -o /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/Firmware/Tools/sitl_gazebo/models/rotors_description`
    1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. Change the vertical angle of the rays in urdf `sed -i 's/_/_/g' /Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. `roslaunch px4 posix_sitl_urdf.launch`

1. New terminal with ros_ws
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. `cd /Firmware`
    1. `source /ros_ws/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo`
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
    


The rqt error is harmless so far. But in case it gets relevent [here is how to fix it](
https://github.com/activatedgeek/docker-videostack/issues/1)


**EUROC setup but with gazebo:**
- Vicon
- Px4 interfacing with ROS [docs](https://dev.px4.io/en/simulation/ros_interface.html)
`roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`

Launch with GDB for headless (?) `make posix_sitl_default gazebo_iris_gdb` from end of [Gazebo + px4 docs](https://dev.px4.io/en/simulation/gazebo.html)

##### Launch file
- Model UAV + hokuyo sensor
- Node auto pilot relay

# Software
<a id='architecture'></a>
### Architecture (ROS)

[Architecture diagram](https://docs.google.com/drawings/d/1fRqLOSLiDlbrC7rG28KdJ4CaBSvM_ozNhlzRuhP2z98/edit?usp=sharing)

Hokuyo driver options:
- [urg_node](http://wiki.ros.org/urg_node) LaserScan messages. Stable. Translate with [laser geometry](http://wiki.ros.org/laser_geometry) 
- [Hokuyo 3d](http://wiki.ros.org/hokuyo3d) PointCloud2 messages. Experimental. 
- For simulation use a Velodyne with 270º visibility.

Nodelets:
- Keep single queue for callbacks and spins
- Avoid serialization associated with sockets

Comms options:
- Making a node w/ a service to keep up with current position because the update rate is 100Hz... Probably benefits from nodelets. The frequency is still pretty high but it's the publisher who controls this. If it becomes an issue [topic_tools/throttle](http://wiki.ros.org/topic_tools/throttle) is an option






<a id='rosMysteries'></a>
### Ros Mysteries

- Catkin_simple
    - How to update the message generation on build package the imports them: update frontiers_msg with `catkin build frontiers`. Tried `cd_add_targets_to_package` but it adds the frontiers_msgs to the targets to install and fails.

<a id='mavros'></a>
### Mavros

- [ROS wiki](http://wiki.ros.org/mavros) 
`roslaunch mavros px4.launch`
- [Off board example](https://dev.px4.io/en/ros/mavros_offboard.html)
- mavlink_adapter 

##### Keeping the OffBoard mode:
[From px4](https://github.com/hamishwillee/px4_user_guide/blob/af0381dcdea989b6679200fe7a1b7c70468b82e5/en/flight_modes/offboard.md): Keep sending SET_POSITION_TARGET_LOCAL_NED messages

[From mavros](https://github.com/mavlink/mavros/search?utf8=%E2%9C%93&q=SET_POSITION_TARGET_LOCAL_NED&type=): This message is a PositionTarget.msg. set_position_target_local_ned() posts to topic local, which is consistent to what the [Off board example](https://dev.px4.io/en/ros/mavros_offboard.html) says.

##### Query param value
```
MPC_XY_VEL_MAX
param show MPC_XY_CRUISE
    Symbols: x = used, + = saved, * = unsaved
    x + MPC_XY_CRUISE [292,580] : 7.0000
     1055 parameters total, 654 used.
rosservice call /mavros/param/set "param_id: 'MPC_XY_CRUISE'
    value:
      integer: 0
      real: 7.0" 
```

<a id='px4'></a>
### px4

Opting out of px4 and mavros internal states as they are poorly documented. Especially in the case of Offboard mode. Instead arrival at waypoint will be checked with position. If trajectory generation is later implemented it will be controlled by time.

<a id='ObstacleAvoidance'></a>
### Obstacle Avoidance

- Check area with a bounding boz for security area
- Use cast rays with lower resolution. Might have some correlation issues with cell id
- Think of octomap as Voronoi Map. Always using a resolution that guarantees safety.

<a id='toImplement'></a>
### To Implement

- Lazy theta Star using unknwon cells
- Coverage calculation. Decide is an unknow cell is covered if it is unreachable. Decide the waypoint that needs to be visited to gather the information.
- Explore the relationship between:
    - the size of the vehicle 
    - the safety distance 
    - resolution needed to find all available paths 
    - number of line of sight checks needs to verify a path.

<a id='experiment'></a>
# Experiment

<a id='Simulation_environment'></a>
### Simulation environment
- Marine environment to make it relatable to Oceans
- Make it large so we can experiment with communication failures (>= 200)
- Actinia size 82m x 78m x 35m
- Check if it is interesting putting all together: RAM occupied by octree + size of path found (+ communications) 

<a id='Steps'></a>
### Steps
1. Set volume of the testbead. No prior knowledge
2. Just press go
3. Feed data into pix4d
4. Show map

<a id='dataset'></a>
### Dataset
- Bag
- Frontier selected
- Octomap build

<a id='What_to_measure'></a>
### What to measure
- Coverage
- Flight time needed in autonomous mode
- Flight time and planning time needed in Pix4D
- Flight time needed in old implementation
- Resolution
- Lazy Theta * iterations (old and new implementations)
- Lazy Theta * computation time for each waypoint (old and new implementation)
- RAM usage (old and new implementation)

<a id='thinkpad'></a>
### Thinkpad

##### How often are frontier cells selected?
- Everytime the old one is reached? 
```
@ gridBenchmark/frontier_node.copp
main()
{
  while(ros::ok())
 {
    mapper::Voxel next_frontier_vector3 = findNextFrontier();
    // ROS_WARN_STREAM("[Frontier] Next frontier " << next_frontier_vector3);
    frontier_pose.pose.position.x = next_frontier_vector3.x;
    frontier_pose.pose.position.y = next_frontier_vector3.y;
    frontier_pose.pose.position.z = next_frontier_vector3.z;
    local_pos_pub.publish(frontier_pose);
    // get_frontier_node = false;
    frontier_check_rate.sleep();
    ros::spinOnce();
 }
}
```
--> This should be on demand

##### Is node topic scope ok?

##### Flying too close to the ground

##### Need better simulation scenario, check how much ground is there.


<a id='LaunchingInstructions'></a>
### Launching Instructions

1. **Launch docker + gazebo + px4 sitl**
    1. `xhost +`
    1. Lauch docker `docker run -it --privileged \
    -v /home/mfaria/docker_px4/Firmware:/Firmware:rw \
    -v /home/mfaria/ws_mavlink_grvcHal/src:/ros_ws/src:rw \
    -v /tmp/.X11-unix:/tmp/.X11-unix:ro \
    -e DISPLAY=:0 \
    --name=garuda_sitl_v2 garuda_sitl_v2 /bin/bash`
    1. exit
    1. `docker start garuda_sitl_v2`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
1. Build ros workspace 
    ```
    source /opt/ros/kinetic/setup.bash
    cd /ros_ws
    catkin build
    ```
1. Build following [px4](https://dev.px4.io/en/simulation/ros_interface.html) instructions - keep terminal
    ```
    cd /Firmware
    source /ros_ws/devel/setup.bash
    source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo
    roslaunch px4 posix_sitl.launch
    ```
1. New terminal
    1. `xhost +`
    1. `docker exec -u 0 -it garuda_sitl_v2 bash`
    1. Setup path
    ```
    cd /Firmware
    source /ros_ws/devel/setup.bash
    source Tools/setup_gazebo.bash /Firmware /Firmware/build/posix_sitl_default
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware
    export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:/Firmware/Tools/sitl_gazebo
    ```
1. Launch mavros node `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`
1. In new terminal, launch offboard node `rosrun architecture px4_mavros_offboard_node`

<a id='Implementation'></a>
# Implementation details
Check from JINT

### Heuristic to goal's voxel center or just goal?

### Line of sight going through explored terrain?

<a id='References'></a>
# References



- Journal of Field Robotics prev implementation
- JINT / ICUAS
- octomap
- gazebo
- ros 
 
 ```@ARTICLE {rotors:2016,
    author  = "Fadri Furrer and Michael Burri and Markus Achtelik and Roland Siegwart",
    title   = "Robot Operating System (ROS)",
    journal = "Studies Comp.Intelligence Volume Number:625",
    year    = "2016",
    volume  = "The Complete Reference (Volume 1)",
    number  = "978-3-319-26052-5",
    pages   = "Chapter 23",
    note    = "ISBN:978-3-319-26052-5"```
}
- tf library 

```@inproceedings {6556373,
    title = {tf: The transform library},
    booktitle = {Technologies for Practical Robot Applications (TePRA), 2013 IEEE International Conference on},
    series = {Open-Source Software workshop},
    year = {2013},
    month={April}, 
    pages={1-6}, 
    author = {Foote, Tully},
    doi={10.1109/TePRA.2013.6556373}, 
    ISSN={2325-0526}
}```

```
@INPROCEEDINGS{7353622,
author={M. Burri and H. Oleynikova and M. W. Achtelik and R. Siegwart},
booktitle={2015 IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS)},
title={Real-time visual-inertial mapping, re-localization and planning onboard MAVs in unknown environments},
year={2015},
volume={},
number={},
pages={1872-1878},
keywords={autonomous aerial vehicles;distance measurement;inertial systems;microrobots;path planning;robot vision;IMU measurements;dynamic 3D paths;globally consistent map;onboard MAV;real-time visual-inertial mapping;unknown environments;visual-inertial odometry;Buildings;Cameras;Helicopters;Inspection;Planning;Real-time systems;Visualization},
doi={10.1109/IROS.2015.7353622},
ISSN={},
month={Sept},}
```